In [1]:
#### Constants in the scheme
C = var("C", latex_name="\\mathcal{C}") ## Courant number
s1 = var("s1", latex_name="s_1")        ## First relaxation parameter (useless)
s2 = var("s2", latex_name="s_2")        ## Second relaxation parameter

assume(C,'real')
assume(s2,'real')

#### Pieces involved in the scheme
q = 2 ## Number of discrete velocities
c1 = 1  ## First discrete velocity
c2 = -1 ## Second discrete velocity
r = 1 ## Stencil to the left
p = 1 ## Stencil to the right
M = matrix([[1,  1], \
            [1, -1]]) ## Moment matrix
eps1 = 1 ## First equilibrium coefficient (useless)
eps2 = C ## Second equilibrium coefficient

#### Construction of the objects involved in the analysis
k = var("kappa", latex_name="\\kappa") 

Minv = M.inverse()
K = identity_matrix(q)+diagonal_matrix([s1, s2])*(matrix([[eps1], [eps2]])*matrix([[1, 0]])-identity_matrix(q))
Ehat = M*diagonal_matrix([k**(-c1), k**(-c2)])*Minv*K #### Bulk matrix scheme "Fourier"
for i in range(q):
    for j in range(q):
        Ehat[i, j] = Ehat[i, j].full_simplify().collect(k)
    
z = var("z", latex_name="z") 

charEq = (z*identity_matrix(q)-Ehat).determinant().full_simplify().collect(k) #### Characteristic equation
dm1 = charEq.coefficient(k, -1).collect(z) ### Coefficient d_{-1} of the characteristic equation
d0  = charEq.coefficient(k,  0).collect(z) ### Coefficient d_{0} of the characteristic equation
dp1 = charEq.coefficient(k,  1).collect(z) ### Coefficient d_{1} of the characteristic equation

Em1 = matrix(SR, q, q) ### Matrix for the point -1
E0  = matrix(SR, q, q) ### Matrix for the point 0
Ep1 = matrix(SR, q, q) ### Matrix for the point 1
for i in range(q):
    for j in range(q):
        Em1[i, j] = Ehat[i, j].coefficient(k, -1)
        E0[i, j]  = Ehat[i, j].coefficient(k,  0)
        Ep1[i, j] = Ehat[i, j].coefficient(k,  1)
Lz = - Em1 + (z*identity_matrix(q)-E0)*k - Ep1*k**2 ### Matrix polynomial in k
print('!! Sanity check: the result must be zero.')
pretty_print((Lz.determinant()/k**(r*q)-charEq).full_simplify()) ### Sanity check

kplus  = ((-d0+sqrt(d0**2-4*dp1*dm1))/2/dp1).full_simplify() ### Root of the char eq with the plus
kminus = ((-d0-sqrt(d0**2-4*dp1*dm1))/2/dp1).full_simplify() ### Root of the char eq with the minus


Pi = dm1/dp1 ### This is the product of the roots in k 

!! Sanity check: the result must be zero.


0

In [2]:
##### Boundary conditions that we analyze for any value of C and s_2
bdMat_BB = z*identity_matrix(q) - M*matrix([[0, 1], [0, k]])*Minv*K ## Bounce back
bdMat_ABB = z*identity_matrix(q) - M*matrix([[0, -1], [0, k]])*Minv*K ## Anti Bounce back
bdMat_TwoABB = z*identity_matrix(q) - M*matrix([[0, -k], [0, k]])*Minv*K ## Two steps Anti Bounce back
bdMat_sigma1 = z*identity_matrix(q) - M*matrix([[1, 0], [0, k]])*Minv*K ## Extrapolation sigma = 1
bdMat_sigma2 = z*identity_matrix(q) - M*matrix([[2-k, 0], [0, k]])*Minv*K ## Extrapolation sigma = 2
bdMat_sigma3 = z*identity_matrix(q) - M*matrix([[3-3*k+k**2, 0], [0, k]])*Minv*K ## Extrapolation sigma = 3
bdMat_sigma4 = z*identity_matrix(q) - M*matrix([[4-6*k+4*k**2-k**3, 0], [0, k]])*Minv*K ## Extrapolation sigma = 4
bdMat_kinDir = z*identity_matrix(q) - M*matrix([[0, 0], [0, k]])*Minv*K ## Kinetic dirichlet

In [3]:
##### We find the critical (z, k) eigenvalues between boundary and bulk
print("Bounce back")
pretty_print(solve([Lz.determinant().full_simplify(), bdMat_BB.determinant().full_simplify()], (z, k)))
print("Anti Bounce back")
pretty_print(solve([Lz.determinant().full_simplify(), bdMat_ABB.determinant().full_simplify()], (z, k)))
print("Two steps anti Bounce back")
pretty_print(solve([Lz.determinant().full_simplify(), bdMat_TwoABB.determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 1")
pretty_print(solve([Lz.determinant().full_simplify(), bdMat_sigma1.determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 2")
pretty_print(solve([Lz.determinant().full_simplify(), bdMat_sigma2.determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 3")
pretty_print(solve([Lz.determinant().full_simplify(), bdMat_sigma3.determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 4")
pretty_print(solve([Lz.determinant().full_simplify(), bdMat_sigma4.determinant().full_simplify()], (z, k)))
print("Kinetic Dirichlet")
pretty_print(solve([Lz.determinant().full_simplify(), bdMat_kinDir.determinant().full_simplify()], (z, k)))

Bounce back


[[z == 0, kappa == 0], [z == -1/2*(C - 1)*s2, kappa == 0], [z == 1, kappa == -((C - 1)*s2 + 2)/((C + 1)*s2 - 2)], [z == s2 - 1, kappa == -1]]

Anti Bounce back


[[z == 0, kappa == 0], [z == 1/2*(C - 1)*s2, kappa == 0], [z == -1, kappa == ((C - 1)*s2 + 2)/((C + 1)*s2 - 2)], [z == -s2 + 1, kappa == 1]]

Two steps anti Bounce back


...

[[z == 3/2*(C - 1)*s2 + 3, kappa == 0], [z == 0, kappa == 0], [z == 1, kappa == 1], [z == -s2 + 1, kappa == 1]]

Extrapolation sigma = 4


[[z == 2*(C - 1)*s2 + 4, kappa == 0], [z == 0, kappa == 0], [z == 1, kappa == 1], [z == -s2 + 1, kappa == 1]]

Kinetic Dirichlet


[[z == 0, kappa == 0]]

In [4]:
##### Sanity check : we set some values for s_2 and C and see if we find more
##### Sometimes sagemath does not give all the roots
s2_val = 2
C_val = -1/2
print("Bounce back")
pretty_print(solve([Lz.subs(s2==s2_val, C==C_val).determinant().full_simplify(), bdMat_BB.subs(s2==s2_val, C==C_val).determinant().full_simplify()], (z, k)))
print("Anti Bounce back")
pretty_print(solve([Lz.subs(s2==s2_val, C==C_val).determinant().full_simplify(), bdMat_ABB.subs(s2==s2_val, C==C_val).determinant().full_simplify()], (z, k)))
print("Two steps anti Bounce back")
pretty_print(solve([Lz.subs(s2==s2_val, C==C_val).determinant().full_simplify(), bdMat_TwoABB.subs(s2==s2_val, C==C_val).determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 1")
pretty_print(solve([Lz.subs(s2==s2_val, C==C_val).determinant().full_simplify(), bdMat_sigma1.subs(s2==s2_val, C==C_val).determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 2")
pretty_print(solve([Lz.subs(s2==s2_val, C==C_val).determinant().full_simplify(), bdMat_sigma2.subs(s2==s2_val, C==C_val).determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 3")
pretty_print(solve([Lz.subs(s2==s2_val, C==C_val).determinant().full_simplify(), bdMat_sigma3.subs(s2==s2_val, C==C_val).determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 4")
pretty_print(solve([Lz.subs(s2==s2_val, C==C_val).determinant().full_simplify(), bdMat_sigma4.subs(s2==s2_val, C==C_val).determinant().full_simplify()], (z, k)))
print("Kinetic Dirichlet")
pretty_print(solve([Lz.subs(s2==s2_val, C==C_val).determinant().full_simplify(), bdMat_kinDir.subs(s2==s2_val, C==C_val).determinant().full_simplify()], (z, k)))

Bounce back


[[z == 0, kappa == 0], [z == (3/2), kappa == 0], [z == 1, kappa == -1]]

Anti Bounce back


[[z == 0, kappa == 0], [z == (-3/2), kappa == 0], [z == -1, kappa == 1]]

Two steps anti Bounce back


...

[[z == (-3/2), kappa == 0], [z == 0, kappa == 0], [z == -1, kappa == 1], [z == 1, kappa == 1]]

Extrapolation sigma = 4


[[z == -2, kappa == 0], [z == 0, kappa == 0], [z == -1, kappa == 1], [z == 1, kappa == 1]]

Kinetic Dirichlet


[[z == 0, kappa == 0]]

In [5]:
#### We make natural assumptions
assume(s2>0)
assume(s2<=2)

In [6]:
#### Critical points (-1, -Pi) and (1, Pi) --- For C<0
assume(C<0)
print("Value of kminus at z=-1")
pretty_print(kminus.subs(z==-1).full_simplify())
print("Value of kplus at z=-1")
pretty_print(kplus.subs(z==-1).full_simplify())
print("Value of kminus at z=1")
pretty_print(kminus.subs(z==1).full_simplify())
print("Value of kplus at z=1")
pretty_print(kplus.subs(z==1).full_simplify())

print("Value of Pi")
pretty_print(Pi)

ks = kplus.full_simplify()

LzOnStable = Lz.subs(k==ks) ### This is the matrix polynomial where k is replaced by ks (stable root)
for i in range(q):
    for j in range(q):
        LzOnStable[i, j] = LzOnStable[i, j].full_simplify()
        
#pretty_print(LzOnStable.transpose().kernel()) #### This is the stable eigenvalue phi_s
phiS = matrix([[LzOnStable.transpose().kernel().basis()[0][0]], \
               [LzOnStable.transpose().kernel().basis()[0][1].full_simplify()]])

#### Structure of the eigenvector close to the critical points
print('Expansion of the second component of phiS around z = -1 as s2 != 2')
pretty_print(taylor(phiS[1][0], z, -1, 1))
print('Expansion of the second component of phiS around z = -1 as s2 = 2')
pretty_print(taylor(phiS[1][0].subs(s2=2), z, -1, 1))
print('Expansion of the second component of phiS around z = 1 as s2 != 2')
pretty_print(taylor(phiS[1][0], z, 1, 1))
print('Expansion of the second component of phiS around z = 1 as s2 = 2')
pretty_print(taylor(phiS[1][0].subs(s2=2), z, 1, 1))

#### We construct the eigenvector of k = 0
phi0 = matrix([[ (s2-1)*Lz.subs(k==0).transpose().kernel().basis()[0][0]], \
               [((s2-1)*Lz.subs(k==0).transpose().kernel().basis()[0][1]).full_simplify()]])

#### We start analyzing critical points and Kreiss Lopatinskii determinants
#### (-1, -Pi)


bd00  = matrix(SR, q, q)
bd01 = matrix(SR, q, q)
bd02 = matrix(SR, q, q)
bd03 = matrix(SR, q, q)
Bs = matrix(SR, q, q) #### Boundary matrix on the stable subspace

#### Bounce back for s_2 = 2
for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_BB[i, j].subs(s2==2).full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_BB[i, j].subs(s2==2).full_simplify().collect(k).coefficient(k, 1)
        
Bs[:, 0] = (bd00 + bd01*ks.subs(s2==2))*phiS.subs(s2==2) ### First column
Bs[:, 1] = (bd00)*phi0.subs(s2==2) ### Second column

for i in range(q):
    for j in range(q):
        Bs[i, j] = Bs[i, j].full_simplify()
KLDet = Bs.determinant().full_simplify()

print("Kreiss Lopatinskii determinant for BB with s2=2 at z=-1 (Taylor expansion)")
pretty_print(taylor(KLDet, z, -1, -1))


#### Anti-bounce back for s_2 in (0, 2)
for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_ABB[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_ABB[i, j].full_simplify().collect(k).coefficient(k, 1)
        
Bs[:, 0] = (bd00 + bd01*ks)*phiS ### First column
Bs[:, 1] = (bd00)*phi0 ### Second column

for i in range(q):
    for j in range(q):
        Bs[i, j] = Bs[i, j].full_simplify()
KLDet = Bs.determinant().full_simplify()

print("Kreiss Lopatinskii determinant for ABB with s2 in (0, 2) at z=-1")
pretty_print(KLDet.subs(z==-1).full_simplify())
print("Boundary matrix on the stable subspace for ABB with s2 in (0, 2) at z=-1 (Taylor expansion)")
KLAdjugate = Bs.adjugate()
for i in range(q):
    print('i = '+str(i))
    for j in range(q):
        print('j = '+str(j))
        pretty_print(taylor(KLAdjugate[i, j]/KLDet, z, -1, -1))
        
print("Computation of the long-time solution with residues")
g0Tilde = matrix([[1], [1]])
var('n') ### Time index
var('j') ### Space index
print('For m_1 with s2 in (0, 2)')
pretty_print(((KLAdjugate/KLDet*g0Tilde)[0][0]*phiS[0][0]*z**(n-1)*ks**(j)).maxima_methods().residue(z,-1).full_simplify())       
print('For m_2 with s2 in (0, 2)')
pretty_print(((KLAdjugate/KLDet*g0Tilde)[0][0]*phiS[1][0]*z**(n-1)*ks**(j)).maxima_methods().residue(z,-1).full_simplify())
print('For m_1 with s2=2')
pretty_print(((KLAdjugate/KLDet*g0Tilde)[0][0]*phiS[0][0]*z**(n-1)*ks**(j)).subs(s2==2).maxima_methods().residue(z,-1).full_simplify())       
print('For m_2 with s2=2')
pretty_print(((KLAdjugate/KLDet*g0Tilde)[0][0]*phiS[1][0]*z**(n-1)*ks**(j)).subs(s2==2).maxima_methods().residue(z,-1).full_simplify())
      
    
#print('For m_1 with s2 in (0, 2)')
#print(latex(((KLAdjugate/KLDet*g0Tilde)[0][0]*phiS[0][0]*z**(n-1)*ks**(j)).maxima_methods().residue(z,-1).full_simplify()))       
#print('For m_2 with s2 in (0, 2)')
#print(latex(((KLAdjugate/KLDet*g0Tilde)[0][0]*phiS[1][0]*z**(n-1)*ks**(j)).maxima_methods().residue(z,-1).full_simplify()))
#print('For m_1 with s2=2')
#print(latex(((KLAdjugate/KLDet*g0Tilde)[0][0]*phiS[0][0]*z**(n-1)*ks**(j)).subs(s2==2).maxima_methods().residue(z,-1).full_simplify()))       
#print('For m_2 with s2=2')
#print(latex(((KLAdjugate/KLDet*g0Tilde)[0][0]*phiS[1][0]*z**(n-1)*ks**(j)).subs(s2==2).maxima_methods().residue(z,-1).full_simplify()))
       
#### Anti-bounce back for s_2=2/(1-C)
phi0Tilde = matrix([[1], [1-2*z]])
for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_ABB[i, j].subs(s2==2/(1-C)).full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_ABB[i, j].subs(s2==2/(1-C)).full_simplify().collect(k).coefficient(k, 1)
        
Bs[:, 0] = (bd00)*phi0Tilde + bd01*phi0.subs(s2==2/(1-C)) ### First column
Bs[:, 1] = (bd00)*phi0.subs(s2==2/(1-C)) ### Second column

for i in range(q):
    for j in range(q):
        Bs[i, j] = Bs[i, j].full_simplify()
KLDet = Bs.determinant().full_simplify()

print("Kreiss Lopatinskii determinant for ABB with s2=2/(1-C) in (0, 2) at z=-1")
pretty_print(KLDet.subs(z==-1).full_simplify())
print("Boundary matrix on the stable subspace for ABB with s2=2/(1-C) at z=-1 (Taylor expansion)")
KLAdjugate = Bs.adjugate()
for i in range(q):
    print('i = '+str(i))
    for j in range(q):
        print('j = '+str(j))
        pretty_print(taylor(KLAdjugate[i, j]/KLDet, z, -1, -1))

        
#### Two steps anti Bounce back for s_2 = 2
for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_TwoABB[i, j].subs(s2==2).full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_TwoABB[i, j].subs(s2==2).full_simplify().collect(k).coefficient(k, 1)
        
Bs[:, 0] = (bd00 + bd01*ks.subs(s2==2))*phiS.subs(s2==2) ### First column
Bs[:, 1] = (bd00)*phi0.subs(s2==2) ### Second column

for i in range(q):
    for j in range(q):
        Bs[i, j] = Bs[i, j].full_simplify()
KLDet = Bs.determinant().full_simplify()

print("Kreiss Lopatinskii determinant for Two steps ABB with s2=2 at z=-1")
pretty_print(KLDet.subs(z==-1))
print("Boundary matrix on the stable subspace for Two steps ABB with s2=2 at z=-1")
pretty_print(Bs.subs(z==-1))


#### Extrapolation sigma = 1 for s_2 = 2
for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_sigma1[i, j].subs(s2==2).full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_sigma1[i, j].subs(s2==2).full_simplify().collect(k).coefficient(k, 1)
        
Bs[:, 0] = (bd00 + bd01*ks.subs(s2==2))*phiS.subs(s2==2) ### First column
Bs[:, 1] = (bd00)*phi0.subs(s2==2) ### Second column

for i in range(q):
    for j in range(q):
        Bs[i, j] = Bs[i, j].full_simplify()
KLDet = Bs.determinant().full_simplify()

print("Kreiss Lopatinskii determinant for Extrapolation sigma = 1  with s2=2 at z=-1")
pretty_print(KLDet.subs(z==-1))
print("Boundary matrix on the stable subspace for Extrapolation sigma = 1  with s2=2 at z=-1")
pretty_print(Bs.subs(z==-1).simplify_full())

print("Computation of the long-time solution with residues")
g0Tilde = matrix([[1], [1]])
var('n') ### Time index
var('j') ### Space index
print('For m_1 with s2=2')
pretty_print(((Bs.inverse()*g0Tilde)[0][0]*phiS[0][0]*z**(n-1)*ks**(j)).subs(s2==2).maxima_methods().residue(z,-1).full_simplify())       
print('For m_2 with s2=2')
pretty_print(((Bs.inverse()*g0Tilde)[0][0]*phiS[1][0]*z**(n-1)*ks**(j)).subs(s2==2).maxima_methods().residue(z,-1).full_simplify())
      
    
#### Extrapolation sigma = 2 for s_2 = 2
for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_sigma2[i, j].subs(s2==2).full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_sigma2[i, j].subs(s2==2).full_simplify().collect(k).coefficient(k, 1)
        bd02[i, j] = bdMat_sigma2[i, j].subs(s2==2).full_simplify().collect(k).coefficient(k, 2)
        
Bs[:, 0] = (bd00 + bd01*ks.subs(s2==2) + bd02*(ks**2).subs(s2==2))*phiS.subs(s2==2) ### First column
Bs[:, 1] = (bd00)*phi0.subs(s2==2) ### Second column

for i in range(q):
    for j in range(q):
        Bs[i, j] = Bs[i, j].full_simplify()
KLDet = Bs.determinant().full_simplify()

print("Kreiss Lopatinskii determinant for Extrapolation sigma = 2  with s2=2 at z=-1 (Taylor)")
pretty_print(taylor(KLDet, z, -1, 2))

BsInv = Bs.inverse()
print("Inverse of the boundary matrix on the stable subspace for Extrapolation sigma = 2  with s2=2 at z=-1 (Taylor)")

for i in range(q):
    print('i = '+str(i))
    for j in range(q):
        print('j = '+str(j))
        pretty_print(taylor(BsInv[i, j], z, -1, 0))
        
print("Computation of the long-time solution with residues")
g0Tilde = matrix([[1], [1]])
var('n') ### Time index
var('j') ### Space index
print('For m_1 with s2=2')
pretty_print(((Bs.inverse()*g0Tilde)[0][0]*phiS[0][0]*z**(n-1)*ks**(j)).subs(s2==2).maxima_methods().residue(z,-1).full_simplify())       
print('For m_2 with s2=2')
pretty_print(((Bs.inverse()*g0Tilde)[0][0]*phiS[1][0]*z**(n-1)*ks**(j)).subs(s2==2).maxima_methods().residue(z,-1).full_simplify().factor())
      
    

#### Extrapolation sigma = 3 for s_2 = 2
for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_sigma3[i, j].subs(s2==2).full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_sigma3[i, j].subs(s2==2).full_simplify().collect(k).coefficient(k, 1)
        bd02[i, j] = bdMat_sigma3[i, j].subs(s2==2).full_simplify().collect(k).coefficient(k, 2)
        
Bs[:, 0] = (bd00 + bd01*ks.subs(s2==2) + bd02*(ks**2).subs(s2==2))*phiS.subs(s2==2) ### First column
Bs[:, 1] = (bd00)*phi0.subs(s2==2) ### Second column

for i in range(q):
    for j in range(q):
        Bs[i, j] = Bs[i, j].full_simplify()
KLDet = Bs.determinant().full_simplify()

print("Kreiss Lopatinskii determinant for Extrapolation sigma = 3  with s2=2 at z=-1 (Taylor)")
pretty_print(taylor(KLDet, z, -1, 2))

BsInv = Bs.inverse()
print("Inverse of the boundary matrix on the stable subspace for Extrapolation sigma = 3  with s2=2 at z=-1 (Taylor)")

for i in range(q):
    print('i = '+str(i))
    for j in range(q):
        print('j = '+str(j))
        pretty_print(taylor(BsInv[i, j], z, -1, 0))
        
print("Computation of the long-time solution with residues")
g0Tilde = matrix([[1], [1]])
var('n') ### Time index
var('j') ### Space index
print('For m_1 with s2=2')
pretty_print(((Bs.inverse()*g0Tilde)[0][0]*phiS[0][0]*z**(n-1)*ks**(j)).subs(s2==2).maxima_methods().residue(z,-1).full_simplify())       
print('For m_2 with s2=2')
pretty_print(((Bs.inverse()*g0Tilde)[0][0]*phiS[1][0]*z**(n-1)*ks**(j)).subs(s2==2).maxima_methods().residue(z,-1).full_simplify().factor())
      
    

#### Extrapolation sigma = 4 for s_2 = 2
for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_sigma4[i, j].subs(s2==2).full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_sigma4[i, j].subs(s2==2).full_simplify().collect(k).coefficient(k, 1)
        bd02[i, j] = bdMat_sigma4[i, j].subs(s2==2).full_simplify().collect(k).coefficient(k, 2)
        bd03[i, j] = bdMat_sigma4[i, j].subs(s2==2).full_simplify().collect(k).coefficient(k, 3)
        
Bs[:, 0] = (bd00 + bd01*ks.subs(s2==2) + bd02*(ks**2).subs(s2==2) + bd03*(ks**3).subs(s2==2))*phiS.subs(s2==2) ### First column
Bs[:, 1] = (bd00)*phi0.subs(s2==2) ### Second column

for i in range(q):
    for j in range(q):
        Bs[i, j] = Bs[i, j].full_simplify()
KLDet = Bs.determinant().full_simplify()

print("Kreiss Lopatinskii determinant for Extrapolation sigma = 4  with s2=2 at z=-1 (Taylor)")
pretty_print(taylor(KLDet, z, -1, 2))

BsInv = Bs.inverse()
print("Inverse of the boundary matrix on the stable subspace for Extrapolation sigma = 4  with s2=2 at z=-1 (Taylor)")

for i in range(q):
    print('i = '+str(i))
    for j in range(q):
        print('j = '+str(j))
        pretty_print(taylor(BsInv[i, j], z, -1, 0))
        
print("Computation of the long-time solution with residues")
g0Tilde = matrix([[1], [1]])
var('n') ### Time index
var('j') ### Space index
print('For m_1 with s2=2')
pretty_print(((Bs.inverse()*g0Tilde)[0][0]*phiS[0][0]*z**(n-1)*ks**(j)).subs(s2==2).maxima_methods().residue(z,-1).full_simplify())       
print('For m_2 with s2=2')
pretty_print(((Bs.inverse()*g0Tilde)[0][0]*phiS[1][0]*z**(n-1)*ks**(j)).subs(s2==2).maxima_methods().residue(z,-1).full_simplify().factor())
      

Value of kminus at z=-1


-1

Value of kplus at z=-1


((C - 1)*s2 + 2)/((C + 1)*s2 - 2)

Value of kminus at z=1


...

-1/2/C

Computation of the long-time solution with residues
For m_1 with s2=2


1/2*(C^3*n^2 - C^3 + C*j^2 + 2*C^2 - (C^2 + 3*C)*j - (C^3 - 2*C^2*j + 3*C^2)*n + 3*C)*(-1)^n

For m_2 with s2=2


-1/6*(2*C^3*n^3 - 6*C^3*n^2 + 6*C^2*j*n^2 - 2*C^3*n - 12*C^2*j*n + 6*C*j^2*n - 9*C^2*n^2 + 6*C^3 - 6*C*j^2 + 2*j^3 + 21*C^2*n - 18*C*j*n - 3*C^2 + 21*C*j - 9*j^2 + 21*C*n - 24*C + 19*j - 15)*(-1)^n*C

In [7]:
#### (1, Pi)

#### Bounce back for s_2 in (0, 2)
for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_BB[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_BB[i, j].full_simplify().collect(k).coefficient(k, 1)
        
Bs[:, 0] = (bd00 + bd01*ks)*phiS ### First column
Bs[:, 1] = (bd00)*phi0 ### Second column

for i in range(q):
    for j in range(q):
        Bs[i, j] = Bs[i, j].full_simplify()
KLDet = Bs.determinant().full_simplify()

print("Kreiss Lopatinskii determinant for BB with s2 in (0, 2) at z=1")
pretty_print(KLDet.subs(z==1).full_simplify())
print("Boundary matrix on the stable subspace for BB with s2 in (0, 2) at z=1 (Taylor expansion)")
KLAdjugate = Bs.adjugate()
for i in range(q):
    print('i = '+str(i))
    for j in range(q):
        print('j = '+str(j))
        pretty_print(taylor(KLAdjugate[i, j]/KLDet, z, 1, -1))
        
print("Computation of the long-time solution with residues")
g0Tilde = matrix([[1], [1]])
var('n') ### Time index
var('j') ### Space index
print('For m_1 with s2 in (0, 2)')
pretty_print(((KLAdjugate/KLDet*g0Tilde)[0][0]*phiS[0][0]*z**(n-1)*ks**(j)).maxima_methods().residue(z,1).full_simplify())       
print('For m_2 with s2 in (0, 2)')
pretty_print(((KLAdjugate/KLDet*g0Tilde)[0][0]*phiS[1][0]*z**(n-1)*ks**(j)).maxima_methods().residue(z,1).full_simplify())
print('For m_1 with s2=2')
pretty_print(((KLAdjugate/KLDet*g0Tilde)[0][0]*phiS[0][0]*z**(n-1)*ks**(j)).subs(s2==2).maxima_methods().residue(z,1).full_simplify().factor())       
print('For m_2 with s2=2')
pretty_print(((KLAdjugate/KLDet*g0Tilde)[0][0]*phiS[1][0]*z**(n-1)*ks**(j)).subs(s2==2).maxima_methods().residue(z,1).full_simplify().factor())
      
    
#### bounce back for s_2=2/(1-C)
phi0Tilde = matrix([[1], [1-2*z]])
for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_BB[i, j].subs(s2==2/(1-C)).full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_BB[i, j].subs(s2==2/(1-C)).full_simplify().collect(k).coefficient(k, 1)
        
Bs[:, 0] = (bd00)*phi0Tilde + bd01*phi0.subs(s2==2/(1-C)) ### First column
Bs[:, 1] = (bd00)*phi0.subs(s2==2/(1-C)) ### Second column

for i in range(q):
    for j in range(q):
        Bs[i, j] = Bs[i, j].full_simplify()
KLDet = Bs.determinant().full_simplify()

print("Kreiss Lopatinskii determinant for BB with s2=2/(1-C) in (0, 2) at z=1")
pretty_print(KLDet.subs(z==1).full_simplify())
print("Boundary matrix on the stable subspace for BB with s2=2/(1-C) at z=1 (Taylor expansion)")
KLAdjugate = Bs.adjugate()
for i in range(q):
    print('i = '+str(i))
    for j in range(q):
        print('j = '+str(j))
        pretty_print(taylor(KLAdjugate[i, j]/KLDet, z, 1, 0))

        

#### Two steps anti Bounce back for s_2 = 2
for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_TwoABB[i, j].subs(s2==2).full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_TwoABB[i, j].subs(s2==2).full_simplify().collect(k).coefficient(k, 1)
        
Bs[:, 0] = (bd00 + bd01*ks.subs(s2==2))*phiS.subs(s2==2) ### First column
Bs[:, 1] = (bd00)*phi0.subs(s2==2) ### Second column

for i in range(q):
    for j in range(q):
        Bs[i, j] = Bs[i, j].full_simplify()
KLDet = Bs.determinant().full_simplify()

print("Kreiss Lopatinskii determinant for Two steps ABB with s2=2 at z=1")
pretty_print(KLDet.subs(z==1))
print("Boundary matrix on the stable subspace for Two steps ABB with s2=2 at z=1")
pretty_print(Bs.subs(z==1))


print("Computation of the long-time solution with residues")
g0Tilde = matrix([[1], [1]])
var('n') ### Time index
var('j') ### Space index
print('For m_1 with s2=2')
pretty_print((((Bs.inverse()*g0Tilde)[0][0]*phiS[0][0]*z**(n-1)*ks**(j)).subs(s2==2).maxima_methods().residue(z,1)+\
              ((Bs.inverse()*g0Tilde)[0][0]*phiS[0][0]*z**(n-1)*ks**(j)).subs(s2==2).maxima_methods().residue(z,-1)).full_simplify().factor())       
print('For m_2 with s2=2')
pretty_print((((Bs.inverse()*g0Tilde)[0][0]*phiS[1][0]*z**(n-1)*ks**(j)).subs(s2==2).maxima_methods().residue(z,1)+\
              ((Bs.inverse()*g0Tilde)[0][0]*phiS[1][0]*z**(n-1)*ks**(j)).subs(s2==2).maxima_methods().residue(z,-1)).full_simplify().factor())  

Kreiss Lopatinskii determinant for BB with s2 in (0, 2) at z=1


0

Boundary matrix on the stable subspace for BB with s2 in (0, 2) at z=1 (Taylor expansion)
i = 0
j = 0


2*C*s2/(((C + 1)*s2 - 2)*(z - 1))

j = 1


...

[      1       1]
[      1 2*C + 1]

Computation of the long-time solution with residues
For m_1 with s2=2


0

For m_2 with s2=2


-2*((-1)^j - (-1)^n)*C

In [8]:
forget()

#### We make natural assumptions
assume(s2>0)
assume(s2<=2)

#### Critical points (1, 1)
assume(C>0)

print("Value of kminus at z=1")
pretty_print(kminus.subs(z==1).full_simplify())
print("Value of kplus at z=1")
pretty_print(kplus.subs(z==1).full_simplify())

ks = kplus.full_simplify()

LzOnStable = Lz.subs(k==ks) ### This is the matrix polynomial where k is replaced by ks (stable root)
for i in range(q):
    for j in range(q):
        LzOnStable[i, j] = LzOnStable[i, j].full_simplify()
        
#pretty_print(LzOnStable.transpose().kernel()) #### This is the stable eigenvalue phi_s
phiS = matrix([[LzOnStable.transpose().kernel().basis()[0][0]], \
               [LzOnStable.transpose().kernel().basis()[0][1].full_simplify()]])

#### Structure of the eigenvector close to the critical points
print('Expansion of the second component of phiS around z = 1')
pretty_print(taylor(phiS[1][0], z, 1, 1)) # I do not know why taking .subs(z==1) gives wrong results
print('Expansion of the second component of phiS around z = 1 as s2 = 2')
pretty_print(taylor(phiS[1][0].subs(s2=2), z, 1, 1))


#### We construct the eigenvector of k = 0
phi0 = matrix([[ (s2-1)*Lz.subs(k==0).transpose().kernel().basis()[0][0]], \
               [((s2-1)*Lz.subs(k==0).transpose().kernel().basis()[0][1]).full_simplify()]])


#### Extrapolation sigma = 1

for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_sigma1[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_sigma1[i, j].full_simplify().collect(k).coefficient(k, 1)
        
Bs[:, 0] = (bd00 + bd01*ks)*phiS ### First column
Bs[:, 1] = (bd00)*phi0 ### Second column

for i in range(q):
    for j in range(q):
        Bs[i, j] = Bs[i, j].full_simplify()
KLDet = Bs.determinant().full_simplify()

print("Kreiss Lopatinskii determinant for Extrapolation sigma = 1 at z=1")
pretty_print(taylor(KLDet, z, 1, 0))
print("Boundary matrix on the stable subspace for Extrapolation sigma = 1  at z=1")
pretty_print(taylor(Bs, z, 1, 0))

print("Computation of the long-time solution with residues")
g0Tilde = matrix([[1], [1]])
var('n') ### Time index
var('j') ### Space index
print('For m_1')
pretty_print(((Bs.inverse()*g0Tilde)[0][0]*phiS[0][0]*z**(n-1)*ks**(j)).maxima_methods().residue(z,1).full_simplify())       
print('For m_2')
pretty_print(((Bs.inverse()*g0Tilde)[0][0]*phiS[1][0]*z**(n-1)*ks**(j)).maxima_methods().residue(z,1).full_simplify())
      
    
#### Extrapolation sigma = 2

for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_sigma2[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_sigma2[i, j].full_simplify().collect(k).coefficient(k, 1)
        
Bs[:, 0] = (bd00 + bd01*ks)*phiS ### First column
Bs[:, 1] = (bd00)*phi0 ### Second column

for i in range(q):
    for j in range(q):
        Bs[i, j] = Bs[i, j].full_simplify()
KLDet = Bs.determinant().full_simplify()

print("Kreiss Lopatinskii determinant for Extrapolation sigma = 2 at z=1")
pretty_print(taylor(KLDet, z, 1, 0))
print("Boundary matrix on the stable subspace for Extrapolation sigma = 2 at z=1")
pretty_print(taylor(Bs, z, 1, 0))

print("Computation of the long-time solution with residues")
g0Tilde = matrix([[1], [1]])
var('n') ### Time index
var('j') ### Space index
print('For m_1')
pretty_print(((Bs.inverse()*g0Tilde)[0][0]*phiS[0][0]*z**(n-1)*ks**(j)).maxima_methods().residue(z,1).full_simplify().factor())       
print('For m_2')
pretty_print(((Bs.inverse()*g0Tilde)[0][0]*phiS[1][0]*z**(n-1)*ks**(j)).maxima_methods().residue(z,1).full_simplify().factor())
      
    
#### Extrapolation sigma = 3

for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_sigma3[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_sigma3[i, j].full_simplify().collect(k).coefficient(k, 1)
        bd02[i, j] = bdMat_sigma3[i, j].full_simplify().collect(k).coefficient(k, 2)
        
Bs[:, 0] = (bd00 + bd01*ks+ bd02*ks**2)*phiS ### First column
Bs[:, 1] = (bd00)*phi0 ### Second column

for i in range(q):
    for j in range(q):
        Bs[i, j] = Bs[i, j].full_simplify()
KLDet = Bs.determinant().full_simplify()

print("Kreiss Lopatinskii determinant for Extrapolation sigma = 3 at z=1")
pretty_print(taylor(KLDet, z, 1, 0))
print("Boundary matrix on the stable subspace for Extrapolation sigma = 3 at z=1")
pretty_print(taylor(Bs, z, 1, 0))

print("Computation of the long-time solution with residues")
g0Tilde = matrix([[1], [1]])
var('n') ### Time index
var('j') ### Space index
print('For m_1')
pretty_print(((Bs.inverse()*g0Tilde)[0][0]*phiS[0][0]*z**(n-1)*ks**(j)).maxima_methods().residue(z,1).full_simplify().factor())       
print('For m_2')
pretty_print(((Bs.inverse()*g0Tilde)[0][0]*phiS[1][0]*z**(n-1)*ks**(j)).maxima_methods().residue(z,1).full_simplify().factor())
      

Value of kminus at z=1


-((C - 1)*s2 + 2)/((C + 1)*s2 - 2)

Value of kplus at z=1


1

Expansion of the second component of phiS around z = 1


...

[       0   s2 - 1]
[       0 C*s2 + 1]

Computation of the long-time solution with residues
For m_1


(C^3*n^2*s2^2 + C^3*n*s2^2 - 2*C^2*j*n*s2^2 - 4*C^3*n*s2 + C*j^2*s2^2 + 3*C^2*n*s2^2 - 4*C^3*s2 + 2*C^2*j*s2 - 2*C^2*n*s2 - 3*C*j*s2^2 - 3*C*n*s2^2 + 6*C^3 - 5*C^2*s2 + 2*C*j*s2 + 6*C*n*s2 + 3*C*s2^2 + 2*j*s2^2 + 4*C^2 + C*s2 - 4*j*s2 - 3*s2^2 - 6*C + 8*s2 - 4)/((C + 1)*s2^2)

For m_2


(C^4*n^2*s2^2 + C^4*n*s2^2 - 2*C^3*j*n*s2^2 - 6*C^4*n*s2 + C^2*j^2*s2^2 + 3*C^3*n*s2^2 - 6*C^4*s2 + 4*C^3*j*s2 - 2*C^3*n*s2 - 3*C^2*j*s2^2 - 3*C^2*n*s2^2 + 12*C^4 - 8*C^3*s2 + 2*C^2*j*s2 + 8*C^2*n*s2 + 3*C^2*s2^2 + 2*C*j*s2^2 + 6*C^3 + 5*C^2*s2 - 6*C*j*s2 - 3*C*s2^2 - 16*C^2 + 11*C*s2 - 6*C - 2*s2 + 4)/((C + 1)*s2^2)

In [9]:
#### We now consider specific values of C and s2, for those boundary conditions which we have not considered so far

bdMat_Ext_Eq_sigma1 = z*identity_matrix(q) - M*matrix([[1/2*(1+C), 1/2*(1+C)], [0, k]])*Minv*K ## Extr eq. sigma=1
bdMat_Ext_Eq_sigma3 = z*identity_matrix(q) - \
          M*matrix([[1/2*(1+C)*(3-3*k+k**2), 1/2*(1+C)*(3-3*k+k**2)], [0, k]])*Minv*K ## Extr eq. sigma=3
bdMat_future = z*identity_matrix(q) - M*matrix([[1/2*(1+C), 1/2*(1+C)*k**2], [0, k]])*Minv*K ## Future
bdMat_invented = z*identity_matrix(q) - M*matrix([[1, 2], [0, k]])*Minv*K ## Invented


for param in [(3/2, -1/2), (2, -1/2), (3/2, 1/2), (2, 1/2)]:
#for param in [(3/2, -1/2)]:

    s2Val = param[0]
    CVal  = param[1]
    
    print('======== s2 = '+str(s2Val)+'   C = '+str(CVal)+' ========')
    
    print('Extrapolated equilibrium sigma = 1')
    roots = solve([charEq.subs(s2==s2Val, C==CVal).full_simplify(), \
                   bdMat_Ext_Eq_sigma1.subs(s2==s2Val, C==CVal).determinant().full_simplify()], [z, k])
    for root in roots:
        pretty_print(root)
        print('abs of z = '+str(abs(root[0].rhs()).n())+'   abs of k = '+str(abs(root[1].rhs()).n()))
        
    print('------')
    print('Extrapolated equilibrium sigma = 3')
    roots = solve([charEq.subs(s2==s2Val, C==CVal).full_simplify(), \
                   bdMat_Ext_Eq_sigma3.subs(s2==s2Val, C==CVal).determinant().full_simplify()], [z, k])
    for root in roots:
        pretty_print(root)
        print('abs of z = '+str(abs(root[0].rhs()).n())+'   abs of k = '+str(abs(root[1].rhs()).n()))
    
    print('------')
    print('Future')
    roots = solve([charEq.subs(s2==s2Val, C==CVal).full_simplify(), \
                   bdMat_future.subs(s2==s2Val, C==CVal).determinant().full_simplify()], [z, k])
    for root in roots:
        pretty_print(root)
        print('abs of z = '+str(abs(root[0].rhs()).n())+'   abs of k = '+str(abs(root[1].rhs()).n()))
        
    print('------')
    print('Invented')
    roots = solve([charEq.subs(s2==s2Val, C==CVal).full_simplify(), \
                   bdMat_invented.subs(s2==s2Val, C==CVal).determinant().full_simplify()], [z, k])
    for root in roots:
        pretty_print(root)
        print('abs of z = '+str(abs(root[0].rhs()).n())+'   abs of k = '+str(abs(root[1].rhs()).n()))

======== s2 = 3/2   C = -1/2 ========
Extrapolated equilibrium sigma = 1


[z == 1, kappa == 1]

abs of z = 1.00000000000000   abs of k = 1.00000000000000


[z == 1/32*sqrt(73) + 3/32, kappa == -1/4*sqrt(73) + 9/4]

abs of z = 0.360750117041173   abs of k = 0.113999063670617


...

abs of z = 1.56155281280883   abs of k = 0.438447187191170


[z == -1/2*sqrt(17) - 1/2, kappa == 1/2*sqrt(17) + 5/2]

abs of z = 2.56155281280883   abs of k = 4.56155281280883


[z == 0, kappa == 0]

abs of z = 0.000000000000000   abs of k = 0.000000000000000


In [34]:
pretty_print(solve(ks.subs(s2==3/2, C==-1/2),z))

[z == -1/2*sqrt(sqrt(16*z^4 - 11*z^2 + 4) + 2), z == 1/2*sqrt(sqrt(16*z^4 - 11*z^2 + 4) + 2)]

In [28]:
pretty_print((bdMat_sigma1.subs(k==0, z==1*(1/2*(C-1)*s2+1))*phi0).simplify_full())
pretty_print((bdMat_sigma2.subs(k==0, z==2*(1/2*(C-1)*s2+1))*phi0).simplify_full())
pretty_print((bdMat_sigma3.subs(k==0, z==3*(1/2*(C-1)*s2+1))*phi0).simplify_full())

[    1/2*(C - 1)*s2^2 - 1/2*(C - 3)*s2 - 1]
[1/2*(C^2 - C)*s2^2 + 1/2*(3*C - 1)*s2 + 1]

[    (C - 1)*s2^2 - (C - 3)*s2 - 2]
[(C^2 - C)*s2^2 + (3*C - 1)*s2 + 2]

[    3/2*(C - 1)*s2^2 - 3/2*(C - 3)*s2 - 3]
[3/2*(C^2 - C)*s2^2 + 3/2*(3*C - 1)*s2 + 3]

In [18]:
pretty_print(phi0)

[  s2 - 1]
[C*s2 + 1]